In [ ]:
# Basic setup and sanity checking
%run ./00_setup.ipynb

## Action Authoring and Deployment
A Cortex Action can be authored and deployed from within a notebook for rapid prototyping and testing.  The Cortex SDK for Python includes an iPython Magic called `%%cortex_action` that takes a notebook cell and deploys it as a Cortex Action.

In [ ]:
%%cortex_action --name c12e/lang-detect --function detect --daemon --requirements langdetect

from cortex import Message
from langdetect import detect_langs


def detect(params):
    msg = Message(params)
    text = msg.payload.get('text')

    results = detect_langs(text)

    others = []
    for lang in results[1:]:
        others.append({'lang': lang.lang, 'score': lang.prob})

    top_lang = results[0]

    return Message.with_payload(
        {'text': text, 'lang': top_lang.lang, 'score': top_lang.prob, 'other_langs': others}).to_params()

## Local Testing
Once the Action is built, we can run both a local and remote test.  The local test in our example can be done by simply calling the `detect` function we defined above.

In [ ]:
params = {'payload': {'text': '¡Bienvenido a CognitiveScale, que tengas un buen día!'}}
rs = detect(params)
result = rs.get('payload')
print(result)
assert(result['lang'] == 'es')

## Remote Testing
Using the Cortex client, we can execute a remote test of our Action to make sure it deployed as expected.

In [ ]:
client = Cortex.client()
action = client.action('c12e/lang-detect')
action

In [ ]:
rs = action.invoke(Message.with_payload(params.get('payload')))
result = rs.payload
print(result)
assert(result['lang'] == 'es')